In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
 os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
config_file = "configs/train_waymo_Copy1.yaml" #train_waymo_Copy1.yaml"#vgg_ssd300_voc0712_tdt4265_server.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2020-04-11 15:26:35,088 SSD INFO: Loaded configuration file configs/train_waymo_Copy1.yaml
2020-04-11 15:26:35,090 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet'   #vgg   #'basic'
        PRETRAINED: True
        OUT_CHANNELS: [512,1024,2048,512, 256, 128]    #[128, 256, 512, 512, 256, 128]
        INPUT_CHANNELS: 3
        DEPTH: 50
INPUT:
    IMAGE_SIZE: 300
DATASETS:
    TRAIN: ("waymo_train",)
    TEST: ("waymo_val", )
SOLVER:
    CHOICE: 'SGD'
    MAX_ITER:  53000
    LR_STEPS: [20000, 400000]
    GAMMA: 0.1
    BATCH_SIZE: 32
    LR: 1.e-2
OUTPUT_DIR: 'outputs/res50_new_waymo_sgd_1e_f2_drop_block' 
DATASET_DIR: "datasets"
2020-04-11 15:26:35,093 SSD INFO: Running with config:
DATASETS:
  TEST: ('waymo_val',)
  TRAIN: ('waymo_train',)
DATASET_DIR: datasets
DATA_LOADER:
  NUM_WORKERS: 8
  PIN_MEMORY: True
EVAL_STEP: 500
INPUT:
  IMAGE_SIZE: 300
  PIXEL_MEAN: [123, 117, 104]
LOG_STEP: 10
MODEL:
  BACKBONE:
    AFTER_TRAINED: False
    DEPTH: 50
    INPUT_C

In [ ]:
model = start_train(cfg)

resnet
Detector initialized. Total Number of params:  39.55M
Backbone number of parameters: 37.52M
SSD Head number of parameters: 2.03M
2020-04-11 15:26:54,148 SSD.trainer INFO: No checkpoint found.
17936
Dataset loaded. Subset: train, number of images: 14348
2020-04-11 15:26:55,689 SSD.trainer INFO: Start training ...
2020-04-11 15:27:25,450 SSD.trainer INFO: iter: 000010, lr: 0.00347, total_loss: 17.760 (17.760), reg_loss: 7.084 (7.084), cls_loss: 10.676 (10.676), time: 2.788 (2.788), eta: 1 day, 17:02:37, mem: 9881M
2020-04-11 15:27:37,779 SSD.trainer INFO: iter: 000020, lr: 0.00360, total_loss: 13.162 (15.461), reg_loss: 6.024 (6.554), cls_loss: 7.138 (8.907), time: 1.233 (2.011), eta: 1 day, 5:35:25, mem: 9881M
2020-04-11 15:27:52,022 SSD.trainer INFO: iter: 000030, lr: 0.00373, total_loss: 10.711 (13.878), reg_loss: 5.568 (6.225), cls_loss: 5.143 (7.652), time: 1.424 (1.815), eta: 1 day, 2:42:31, mem: 9881M
2020-04-11 15:28:06,027 SSD.trainer INFO: iter: 000040, lr: 0.00387, tota

In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)